In [1]:
APP_NAME = "Download data to local machine"


In [2]:
# %load configuration.py
configuration = {
    "pg_url": "jdbc:postgresql://localhost:5432/honeypot",
    "pg_host": "localhost",
    "pg_port": "5432",
    "pg_database": "honeypot",
    "pg_user": "max",
    "pg_password": "kM9ZhBOBFIl",
    "spark_host": "10.10.10.80",
    "spark_uri": "spark://10.10.10.80:7077",
}


In [3]:
# %load data.py


from dateutil import parser

dates = [
    "2021-05-17", #  0: first monday to start logging on
    "2021-05-18", #  1:
    "2021-05-19", #  2:
    "2021-05-20", #  3:
    "2021-05-21", #  4
    "2021-05-22", #  5: Saturday
    "2021-05-23", #  6: Sunday
    "2021-05-24", #  7: Monday (Whit monday)
    "2021-05-25", #  8:
    "2021-05-26", #  9:
    "2021-05-27", # 10: 
    "2021-05-28", # 11:
    "2021-05-29", # 12: Saturday
    "2021-05-30", # 13: Sunday
    "2021-05-31", # 14: first dayout logging
]

dates = [parser.parse(d) for d in dates]

# Get the date filters per week day.
# Consider that each filter consists of a tuple (lower & upper bound)

def get_i_weekday(day: int):
    """Returns a list of indices which represent the corresponding week day."""
    b = day % 7
    return [b, b + 7]

def get_name_weekday(day: int) -> str:
    """Returns the name of the weekday."""
    if day == 0:
        return "Monday"
    elif day == 1:
        return "Tuesday"
    elif day == 2:
        return "Wednesday"
    elif day == 3:
        return "Thursday"
    elif day == 4:
        return "Friday"
    elif day == 5:
        return "Saturday"
    elif day == 6:
        return "Sunday"
    else:
        raise ValueError(f"Invalid day of the week {day}")
    
    
# honeypots

honeypots = [
    ('campus', 1),
    ('campus', 2),
    ('campus', 3),
    ('residential', 6),
    ('residential', 7),
    ('residential', 8),
    ('residential', 9),
    ('cloud', 12),
    ('cloud', 13),
    ('cloud', 14),
    ('cloud', 15),
]

# categories
categories = ['ssh', 'telnet']

In [4]:
import pandas as pd
import numpy as np
import psycopg2

dbc = psycopg2.connect(
    database=configuration['pg_database'],
    host=configuration['pg_host'],
    port=configuration['pg_port'],
    user=configuration['pg_user'], 
    password=configuration['pg_password'],
)

print("Loading 'log' into dataframe")
df_log = pd.read_sql_query(
    f"SELECT * FROM log where '{dates[0]}' <= timestamp AND timestamp < '{dates[-1]}'",
    con=dbc,
    index_col=['id'],
)

print(f"Storing 'log' dataframe in CSV {csv_log}")
df_log.to_csv(csv_log)
print("Successfully stored CSV")

print("Loading 'dataplane' into dataframe")
df_dataplane = pd.read_sql_query(
    f"SELECT * FROM dataplane where '{dates[0]}' <= timestamp AND timestamp < '{dates[-1]}'",
    con=dbc,
    index_col=['id']
)

print(f"Storing 'dataplane' dataframe in CSV {csv_dataplane}")
df_dataplane.to_csv(csv_dataplane)
print("Successfully stored CSV")


print("Loading 'ip' into dataframe")
df_ip = pd.read_sql_query(
    f"SELECT * FROM ip;",
    con=dbc,
    index_col=['id'],
)

print(f"Storing 'log' dataframe in CSV {csv_ip}")
df_ip.to_csv(csv_ip)
print("Successfully stored CSV")

Loading 'log' into dataframe
Storing 'log' dataframe in CSV /home/max/jupyter/dataplane/raw/log.csv
Successfully stored CSV
Loading 'dataplane' into dataframe
Storing 'dataplane' dataframe in CSV /home/max/jupyter/dataplane/raw/dataplane.csv
Successfully stored CSV
Loading 'ip' into dataframe
Storing 'log' dataframe in CSV /home/max/jupyter/dataplane/raw/ip.csv
Successfully stored CSV


# Creation of dataframes

The following section first generates the filter on which the log data needs
to be filtered. Afterwards, the dataframe is created. The data frames are
filtered per honeypot, per distinct IP addresses and number of requests in
total. Also, the data is represented cummulative and as a partial filter per
slot. It is only data shown based on SSH requests and on Telnet requests.

In [5]:
df_log.head()

df_dataplane.head()

,sync_ts,timestamp,asn,asname,category,ip
id,,,,,,
1,2021-05-19 23:45:01+00:00,2021-05-19 22:21:31+00:00,3,MIT-GATEWAYS,ssh,18.27.197.252
2,2021-05-19 23:45:01+00:00,2021-05-19 22:59:11+00:00,137,ASGARR Consortium GARR,ssh,131.114.98.64
5,2021-05-19 23:45:01+00:00,2021-05-19 04:25:38+00:00,174,COGENT-174,ssh,185.142.236.34
6,2021-05-19 23:45:01+00:00,2021-05-19 12:17:34+00:00,174,COGENT-174,ssh,185.142.236.35
7,2021-05-19 23:45:01+00:00,2021-05-19 01:08:44+00:00,174,COGENT-174,ssh,185.142.236.36


In [6]:
"""
# Generate all Filters
# ================

# First define the time deltas (in minutes) which you would like to display
# at the end of this notebook.

#deltas = [3, 5, 10, 15, 30, 60]
deltas = [60, 120]

# The remainin code generates the filters based on the deltas given above.

from datetime import timedelta

# Time delta in Minutes:

deltas = [timedelta(minutes=d) for d in deltas]
log_start = dates[1]
log_end = dates[5]

# filters partially for each partition
filters_par = {}
# filters cumulative from beginning to end for each parition
filters_cum = {}

# format
fmt = '%Y-%m-%d %H:%M:%S'

for d in deltas:
    filter_par = []
    filter_cum = []
    
    x = log_start + d
    filter_par.append(
        (format(log_start, fmt), format(x, fmt))
    )
    
    while x < log_end:
        filter_par.append(
            (format(x, fmt), format(x + d, fmt))
        )
        filter_cum.append(
            (format(log_start, fmt), format(x, fmt))
        )
        
        x += d
    
    filters_par[d] = filter_par
    filters_cum[d] = filter_cum
    
    """

"\n# Generate all Filters\n# ================\n\n# First define the time deltas (in minutes) which you would like to display\n# at the end of this notebook.\n\n#deltas = [3, 5, 10, 15, 30, 60]\ndeltas = [60, 120]\n\n# The remainin code generates the filters based on the deltas given above.\n\nfrom datetime import timedelta\n\n# Time delta in Minutes:\n\ndeltas = [timedelta(minutes=d) for d in deltas]\nlog_start = dates[1]\nlog_end = dates[5]\n\n# filters partially for each partition\nfilters_par = {}\n# filters cumulative from beginning to end for each parition\nfilters_cum = {}\n\n# format\nfmt = '%Y-%m-%d %H:%M:%S'\n\nfor d in deltas:\n    filter_par = []\n    filter_cum = []\n    \n    x = log_start + d\n    filter_par.append(\n        (format(log_start, fmt), format(x, fmt))\n    )\n    \n    while x < log_end:\n        filter_par.append(\n            (format(x, fmt), format(x + d, fmt))\n        )\n        filter_cum.append(\n            (format(log_start, fmt), format(x, fmt))\n 

In [7]:
# Generate dataframes per honeypot and category
# ================
# Load the dataframe to a Pandas DF
# Since the DB is considerably small (< 1 GB) , we hope that the use of
# pandas will speed up the data anlysis.
"""
df_log.dtypes


#pdf = df_log.toPandas()

#keys = []
#dfs = {}

#for h in honeypots:
#    keys += [(cat, h[0], h[1]) for cat in categories]
    

#for k in keys:
#    df = df_log.select(l
#        'id',
#        'category',
#        'origin',
#        'origin_id',
#        'timestamp',
#        'ip'
#    ).filter(
#        f"category == '{k[0]}'"
#    ).filter(
#        f"origin == '{k[1]}'"
#    ).filter(
#        f"origin_id == '{k[2]}'"
#    ).filter(
#        f"'{format(log_start, fmt)}' <= timestamp and timestamp < '{format(log_end, fmt)}'"
#    )
#        
#    dfs[k] = df
#
"""

'\ndf_log.dtypes\n\n\n#pdf = df_log.toPandas()\n\n#keys = []\n#dfs = {}\n\n#for h in honeypots:\n#    keys += [(cat, h[0], h[1]) for cat in categories]\n    \n\n#for k in keys:\n#    df = df_log.select(l\n#        \'id\',\n#        \'category\',\n#        \'origin\',\n#        \'origin_id\',\n#        \'timestamp\',\n#        \'ip\'\n#    ).filter(\n#        f"category == \'{k[0]}\'"\n#    ).filter(\n#        f"origin == \'{k[1]}\'"\n#    ).filter(\n#        f"origin_id == \'{k[2]}\'"\n#    ).filter(\n#        f"\'{format(log_start, fmt)}\' <= timestamp and timestamp < \'{format(log_end, fmt)}\'"\n#    )\n#        \n#    dfs[k] = df\n#\n'

In [8]:
import numpy as np
import pandas as pd
"""
# TODO: Rewrite this part...
# Dataframe cummulative & partial
# The outermost container of the dict is the timedelta.
# In the time delta is the key "category, network type, honeypot" wrapped.
# The innermost layer contains the values per timeslot (depending on the time delta)
df_req_par = {}
df_req_cum = {}

for d in deltas:
    for f in filters_par[d]:       
        df = df_log.select(
            'category','origin', 'origin_id', 'id'
        ).filter(
            f"'{f[0]}' <= timestamp AND timestamp < '{f[1]}'"
        ).groupBy('category', 'origin', 'origin_id').count().withColumn(
            "timestamp", fn.lit(f[0])
        ).select(
            'timestamp', 'category', 'origin', 'origin_id', 'count'
        ).orderBy(
            'timestamp', 'category', 'origin', 'origin_id'
        )
        
        if not df_req_par.get(d):
            df_req_par[d] = df
        else:
            df_req_par[d] = df_req_par[d].union(df)
        
    print(f'Finished partial evaluation on time delta {str(d)}')

    for f in filters_cum[d]:
        df = df_log.select(
            'category', 'origin', 'origin_id', 'id'
        ).filter(
            f"'{f[0]}' <= timestamp AND timestamp < '{f[1]}'"
        ).groupBy('category', 'origin', 'origin_id').count().withColumn(
            'timestamp', fn.lit(f[0])
        ).select(
            'timestamp', 'category', 'origin', 'origin_id', 'count'
        ).orderBy(
            'timestamp', 'category', 'origin', 'origin_id'
        )
        
        if not df_req_cum.get(d):
            df_req_cum[d] = df
        else:
            df_req_cum[d] = df_req_cum[d].union(df)
        
    print(f'Finished cumulative evaluation on time delta {str(d)}')

    
for d in deltas:
    print(f"df_req_par[{d}]: # {df_req_par[d].count()}")
    print(f"df_req_cum[{d}]: # {df_req_cum[d].count()}")
    
    """

'\n# TODO: Rewrite this part...\n# Dataframe cummulative & partial\n# The outermost container of the dict is the timedelta.\n# In the time delta is the key "category, network type, honeypot" wrapped.\n# The innermost layer contains the values per timeslot (depending on the time delta)\ndf_req_par = {}\ndf_req_cum = {}\n\nfor d in deltas:\n    for f in filters_par[d]:       \n        df = df_log.select(\n            \'category\',\'origin\', \'origin_id\', \'id\'\n        ).filter(\n            f"\'{f[0]}\' <= timestamp AND timestamp < \'{f[1]}\'"\n        ).groupBy(\'category\', \'origin\', \'origin_id\').count().withColumn(\n            "timestamp", fn.lit(f[0])\n        ).select(\n            \'timestamp\', \'category\', \'origin\', \'origin_id\', \'count\'\n        ).orderBy(\n            \'timestamp\', \'category\', \'origin\', \'origin_id\'\n        )\n        \n        if not df_req_par.get(d):\n            df_req_par[d] = df\n        else:\n            df_req_par[d] = df_req_par[

In [9]:
"""
pdf_req_par = {}
pdf_req_cum = {}

for d in deltas:
    pdf_req_par[d] = df_req_par[d].toPandas()
    pdf_req_cum[d] = df_req_cum[d].toPandas()
    
"""

'\npdf_req_par = {}\npdf_req_cum = {}\n\nfor d in deltas:\n    pdf_req_par[d] = df_req_par[d].toPandas()\n    pdf_req_cum[d] = df_req_cum[d].toPandas()\n    \n'

In [10]:
from matplotlib import pyplot as plt